In [6]:
from getpass import getpass
from coreapi.auth import BasicAuthentication
from quantuminspire.api import QuantumInspireAPI
from quantuminspire.credentials import load_account, get_token_authentication, get_basic_authentication
from quantuminspire.api import QuantumInspireAPI
from qiskit import IBMQ, QuantumRegister, ClassicalRegister, QuantumCircuit, assemble, transpile
from qiskit.providers.ibmq import least_busy
from qiskit.circuit import Parameter 



import numpy as np
import pandas as pd
import plotly.express as px

In [7]:
#https://stackoverflow.com/questions/9600801/evenly-distributing-n-points-on-a-sphere
#Create n evenly distributed points on a sphere

num_pts = 1000
indices = np.arange(0, num_pts, dtype=float) + 0.5

theta = np.arccos(1 - 2*indices/num_pts)
phi = (np.pi * (1 + 5**0.5) * indices)%(2*np.pi)
coords=[]

for theta_elem,phi_elem in zip(theta,phi):
    coords.append((theta_elem,phi_elem))

x, y, z = np.cos(phi) * np.sin(theta), np.sin(phi) * np.sin(theta), np.cos(theta);

# Plot the surface.
df = px.data.iris()
fig = px.scatter_3d(df, x, y, z, template="plotly_white")

fig.show()
target_points=np.array(coords)
# save the data, optional
np.savetxt("target_points_ibmq_santiago.csv",target_points)

In [8]:
theta_param, phi_param = Parameter('theta_param'), Parameter('phi_param') 

qreg_q = QuantumRegister(3, 'q')
creg_c = ClassicalRegister(3, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.ry(theta_param, qreg_q[0])
circuit.rz(phi_param, qreg_q[0])
    
circuit.ry(1.107149, qreg_q[1])
circuit.cx(qreg_q[1], qreg_q[2])
circuit.ry(0.729728, qreg_q[2])
circuit.cx(qreg_q[2], qreg_q[1])
circuit.ry(0.463648, qreg_q[1])
circuit.cx(qreg_q[0], qreg_q[1])
circuit.cx(qreg_q[0], qreg_q[2])
circuit.cx(qreg_q[1], qreg_q[0])
circuit.cx(qreg_q[2], qreg_q[0])

circuit.rz(-phi_param, qreg_q[0])
circuit.ry(-theta_param, qreg_q[0])

circuit.measure(qreg_q[0], creg_c[0])

In [9]:
IBMQ.save_account('')

N_shots=8192
provider = IBMQ.load_account()

#backend = least_busy(provider.backends(simulator=False))
#print(backend)
backend = provider.backends.ibmq_santiago

configrc.store_credentials:WARNING:2020-11-28 09:42:52,307: Credentials already present. Set overwrite=True to overwrite.
ibmqfactory.load_account:WARNING:2020-11-28 09:42:52,851: Credentials are already in use. The existing account in the session will be replaced.


In [4]:
runningjobs=[]
job_ids=[]
index=0

while index*75 < num_pts:
    if((index+1)*75<num_pts):
        circuits = [circuit.bind_parameters(
        {theta_param: points[0], phi_param: points[1]}) 
         for points in coords[index*75:(index+1)*75]]
        qobj = assemble(transpile(circuits, backend=backend), backend=backend, shots=N_shots)
        runningjobs.append(backend.run(qobj))
        job_ids.append(runningjobs[-1].job_id())
        with open('job_ids_santiago.txt', 'a') as file:
            file.write(runningjobs[-1].job_id()+"\n")
        if(backend.job_limit().active_jobs == backend.job_limit().maximum_jobs):
            print("Maximum number of jobs reached. Waiting...")
            try:
                job_result = runningjobs[0].result()  # It will block until the job finishes.
                print("The job finished with result {}".format(job_result))
                runningjobs.pop(0)
            except JobError as ex:
                print("Something wrong happened!: {}".format(ex))
    else:
        circuits = [circuit.bind_parameters(
        {theta_param: points[0], phi_param: points[1]}) 
         for points in coords[index*75:]]
        qobj = assemble(transpile(circuits, backend=backend), backend=backend, shots=N_shots)
        runningjobs.append(backend.run(qobj))
        job_ids.append(runningjobs[-1].job_id())
        with open('job_ids_santiago.txt', 'a') as file:
            file.write(runningjobs[-1].job_id()+"\n")
        if(backend.job_limit().active_jobs == backend.job_limit().maximum_jobs):
            print("Maximum number of jobs reached. Waiting...")
            try:
                job_result = runningjobs[0].result()  # It will block until the job finishes.
                print("The job finished with result {}".format(job_result))
                runningjobs.pop(0)
            except JobError as ex:
                print("Something wrong happened!: {}".format(ex))
    index=index+1

configrc.store_credentials:WARNING:2020-11-27 18:48:32,488: Credentials already present. Set overwrite=True to overwrite.
/Users/francescozatelli/intelpython3/lib/python3.7/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning:

Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.



Maximum number of jobs reached. Waiting...
The job finished with result Result(backend_name='ibmq_santiago', backend_version='1.2.0', qobj_id='b0c6248f-9c04-44c5-a261-28bbe59b382a', job_id='5fc13bf65368b30019f2bf2d', success=True, results=[ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 6515, '0x1': 1677}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=0.0, memory_slots=3, n_qubits=5, name='circuit7', qreg_sizes=[['q', 5]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]]), memory=False), ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 6638, '0x1': 1554}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=0.0, memory_slots=3, n_qubits=5, name='circuit7', qreg_sizes=[['q', 5]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]]), memory=Fals

Maximum number of jobs reached. Waiting...
The job finished with result Result(backend_name='ibmq_santiago', backend_version='1.2.0', qobj_id='2d23ea4d-a23e-488c-9bed-1b311ffd05b4', job_id='5fc13bfa176371001a21f72b', success=True, results=[ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 6326, '0x1': 1866}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=0.0, memory_slots=3, n_qubits=5, name='circuit7', qreg_sizes=[['q', 5]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]]), memory=False), ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 6354, '0x1': 1838}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=0.0, memory_slots=3, n_qubits=5, name='circuit7', qreg_sizes=[['q', 5]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]]), memory=Fals

Maximum number of jobs reached. Waiting...
The job finished with result Result(backend_name='ibmq_santiago', backend_version='1.2.0', qobj_id='851036d2-2021-46e5-91f6-08ee30a9817d', job_id='5fc13bfed3b367001ad60899', success=True, results=[ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 6065, '0x1': 2127}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=0.0, memory_slots=3, n_qubits=5, name='circuit7', qreg_sizes=[['q', 5]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]]), memory=False), ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 6097, '0x1': 2095}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=0.0, memory_slots=3, n_qubits=5, name='circuit7', qreg_sizes=[['q', 5]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]]), memory=Fals

Maximum number of jobs reached. Waiting...
The job finished with result Result(backend_name='ibmq_santiago', backend_version='1.2.0', qobj_id='ce8a292d-6d82-4699-82a9-64fb018ac345', job_id='5fc13c02e15c3b00192d1ac1', success=True, results=[ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 6036, '0x1': 2156}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=0.0, memory_slots=3, n_qubits=5, name='circuit7', qreg_sizes=[['q', 5]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]]), memory=False), ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 6166, '0x1': 2026}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=0.0, memory_slots=3, n_qubits=5, name='circuit7', qreg_sizes=[['q', 5]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]]), memory=Fals

...

Maximum number of jobs reached. Waiting...
The job finished with result Result(backend_name='ibmq_santiago', backend_version='1.2.0', qobj_id='871102c9-0ae0-4e99-8ad5-497661689d87', job_id='5fc13e9f1dde84001b4c7239', success=True, results=[ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 5872, '0x1': 2320}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=0.0, memory_slots=3, n_qubits=5, name='circuit7', qreg_sizes=[['q', 5]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]]), memory=False), ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 6014, '0x1': 2178}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=0.0, memory_slots=3, n_qubits=5, name='circuit7', qreg_sizes=[['q', 5]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]]), memory=Fals

Maximum number of jobs reached. Waiting...
The job finished with result Result(backend_name='ibmq_santiago', backend_version='1.2.0', qobj_id='5d2bf041-09a5-4e14-ad96-8e344afc8cc0', job_id='5fc13ffcd3b367001ad608c2', success=True, results=[ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 5990, '0x1': 2202}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=0.0, memory_slots=3, n_qubits=5, name='circuit7', qreg_sizes=[['q', 5]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]]), memory=False), ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 5727, '0x1': 2465}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=0.0, memory_slots=3, n_qubits=5, name='circuit7', qreg_sizes=[['q', 5]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]]), memory=Fals

Maximum number of jobs reached. Waiting...
The job finished with result Result(backend_name='ibmq_santiago', backend_version='1.2.0', qobj_id='c47d22e3-03e8-4bf6-8880-9e387c2005fa', job_id='5fc1411145902b0019841757', success=True, results=[ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 6324, '0x1': 1868}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=0.0, memory_slots=3, n_qubits=5, name='circuit7', qreg_sizes=[['q', 5]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]]), memory=False), ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 6347, '0x1': 1845}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=0.0, memory_slots=3, n_qubits=5, name='circuit7', qreg_sizes=[['q', 5]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]]), memory=Fals

Maximum number of jobs reached. Waiting...
The job finished with result Result(backend_name='ibmq_santiago', backend_version='1.2.0', qobj_id='5564fa48-9a68-4bf6-886f-98efb7daf8f1', job_id='5fc14228e15c3b00192d1b01', success=True, results=[ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 6329, '0x1': 1863}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=0.0, memory_slots=3, n_qubits=5, name='circuit7', qreg_sizes=[['q', 5]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]]), memory=False), ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 6423, '0x1': 1769}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=0.0, memory_slots=3, n_qubits=5, name='circuit7', qreg_sizes=[['q', 5]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]]), memory=Fals

Maximum number of jobs reached. Waiting...
The job finished with result Result(backend_name='ibmq_santiago', backend_version='1.2.0', qobj_id='77559134-a9c6-4ff7-bea1-8d20986a6024', job_id='5fc1434045902b0019841769', success=True, results=[ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 6501, '0x1': 1691}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=0.0, memory_slots=3, n_qubits=5, name='circuit7', qreg_sizes=[['q', 5]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]]), memory=False), ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 6081, '0x1': 2111}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=0.0, memory_slots=3, n_qubits=5, name='circuit7', qreg_sizes=[['q', 5]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]]), memory=Fals

In [13]:
read_job_ids = open("job_ids_santiago.txt").read().splitlines()
counts_array=[]
results_probabilities=[]
retrieved_jobs=[backend.retrieve_job(jobid) for jobid in read_job_ids]
for job in retrieved_jobs:
    counts_array=counts_array+job.result().get_counts()
for job in counts_array:
    results_probabilities.append((job["000"]/N_shots,job["001"]/N_shots))
results_probabilities=np.array(results_probabilities)

In [14]:
data_sheet=pd.DataFrame(data=np.hstack((target_points,results_probabilities)),columns=["θ","ϕ","prob_0","prob_1"])

In [15]:
data_sheet.head()

,θ,ϕ,prob_0,prob_1
0,0.044725,5.083204,0.795288,0.204712
1,0.077479,2.683240,0.810303,0.189697
2,0.100042,0.283277,0.793335,0.206665
3,0.118391,4.166499,0.805176,0.194824
4,0.134265,1.766536,0.814331,0.185669


In [16]:
for (point,probabilities) in zip(target_points,results_probabilities):
    with open('results_ibm_santiago.txt', 'a') as file:
        file.write(str(point[0])+"\t"+str(point[1])+"\t"+str(probabilities[0])+"\t"+str(probabilities[1])+"\n")

In [17]:
data_sheet=pd.DataFrame(data=np.hstack((target_points,results_probabilities)),columns=["θ","ϕ","prob_0","prob_1"])

In [18]:
data_sheet.head()

,θ,ϕ,prob_0,prob_1
0,0.044725,5.083204,0.795288,0.204712
1,0.077479,2.683240,0.810303,0.189697
2,0.100042,0.283277,0.793335,0.206665
3,0.118391,4.166499,0.805176,0.194824
4,0.134265,1.766536,0.814331,0.185669


In [19]:
# save to excel, optional
data_sheet.to_excel("data_sheet_ibm_santiago.xlsx")

In [20]:
data_sheet=pd.read_excel("data_sheet_ibm_santiago.xlsx")

In [21]:
# Make data.
thetas=data_sheet["θ"]
phis=data_sheet["ϕ"]
x, y, z = np.cos(phis) * np.sin(thetas), np.sin(phis) * np.sin(thetas), np.cos(thetas)
# Plot the surface. F_measured/F_theory
df = px.data.iris()
fig = px.scatter_3d(df, x, y, z, color=data_sheet.prob_0/(5/6), template="plotly_white")

fig.show()
print("Number of points: ", len(x))
print("Average fidelity: ", np.average(data_sheet.prob_0))
print("Standard deviation: ", np.std(data_sheet.prob_0))


Number of points:  1000
Average fidelity:  0.74606884765625
Standard deviation:  0.05076161201233923
